In [74]:
import os
import sys
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://query.wikidata.org/sparql"

bdd = []

query = """
SELECT ?item ?itemLabel ?pic
{
  ?item wdt:P31 wd:Q144.
  ?item wdt:P18 ?pic
}
limit 100"""


def get_results(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

#Pour creer le fichier des images s'il existe pas
def createFolder(name='images'):
    if name not in os.listdir('./PROJET'):
        os.mkdir('./PROJET/'+name)
    else:
        print('deja crée')

#Pour ajouter des elements a la bdd
def addBDD(url=''):
    results = get_results(endpoint_url, query)

    for result in results:
        
        bdd.append(result)
    print(imgs)

#Pour telecharger l'image d'un element de la bd
def downloadImage(item=''):
    
    pass
    
    
    
        
downloadImg()

[{'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q86516809'}, 'pic': {'type': 'uri', 'value': 'http://commons.wikimedia.org/wiki/Special:FilePath/DetailofTheImperialPrinceandhisdogNerobyJean-BaptisteCarpeaux1865MarbleMuseed%27Orsaymharrsch.jpg'}}, {'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q87783199'}, 'pic': {'type': 'uri', 'value': 'http://commons.wikimedia.org/wiki/Special:FilePath/Photograph%20of%20Thunder%2C%20dog%20star%20of%20the%20silent%20era%2C.jpeg'}}, {'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q96176417'}, 'pic': {'type': 'uri', 'value': 'http://commons.wikimedia.org/wiki/Special:FilePath/Wasao.jpg'}}, {'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q96372991'}, 'pic': {'type': 'uri', 'value': 'http://commons.wikimedia.org/wiki/Special:FilePath/20190727-DSC00957%20%2848391674722%29.jpg'}}, {'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q97280926'}, 'pic': {'type': 'uri', 'va

{'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q86516809'}, 'pic': {'type': 'uri', 'value': 'http://commons.wikimedia.org/wiki/Special:FilePath/DetailofTheImperialPrinceandhisdogNerobyJean-BaptisteCarpeaux1865MarbleMuseed%27Orsaymharrsch.jpg'}}
{'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q87783199'}, 'pic': {'type': 'uri', 'value': 'http://commons.wikimedia.org/wiki/Special:FilePath/Photograph%20of%20Thunder%2C%20dog%20star%20of%20the%20silent%20era%2C.jpeg'}}
{'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q96176417'}, 'pic': {'type': 'uri', 'value': 'http://commons.wikimedia.org/wiki/Special:FilePath/Wasao.jpg'}}
{'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q96372991'}, 'pic': {'type': 'uri', 'value': 'http://commons.wikimedia.org/wiki/Special:FilePath/20190727-DSC00957%20%2848391674722%29.jpg'}}
{'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q97280926'}, 'pic': {'type': 'uri', 'value':

{'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q86516809'}, 'pic': {'type': 'uri', 'value': 'http://commons.wikimedia.org/wiki/Special:FilePath/DetailofTheImperialPrinceandhisdogNerobyJean-BaptisteCarpeaux1865MarbleMuseed%27Orsaymharrsch.jpg'}}
{'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q87783199'}, 'pic': {'type': 'uri', 'value': 'http://commons.wikimedia.org/wiki/Special:FilePath/Photograph%20of%20Thunder%2C%20dog%20star%20of%20the%20silent%20era%2C.jpeg'}}
{'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q96176417'}, 'pic': {'type': 'uri', 'value': 'http://commons.wikimedia.org/wiki/Special:FilePath/Wasao.jpg'}}
{'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q96372991'}, 'pic': {'type': 'uri', 'value': 'http://commons.wikimedia.org/wiki/Special:FilePath/20190727-DSC00957%20%2848391674722%29.jpg'}}
{'item': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q97280926'}, 'pic': {'type': 'uri', 'value':